## Preprocessando SINAN raw_data

### Dependências

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from datetime import datetime

### Carregando dados

In [ ]:
SINAN_RAW_PATH = './../data/raw_data/2. SINAN (violencia sexual)/DadosViolSexualDomestica_2013-18_SINAN.csv'

In [ ]:
%%time

# Apenas para testes (ler apenas as primeiras nrows linhas)
# nrows = 100000

# raw_dataframe = pd.read_csv(SINAN_RAW_PATH, nrows = nrows)
raw_dataframe = pd.read_csv(SINAN_RAW_PATH)

print('Número de colunas:', len(raw_dataframe.columns))
print('Número de linhas:', len(raw_dataframe))

In [ ]:
# Filtrando para estado de SP

uf_id = 35

raw_dataframe = raw_dataframe[raw_dataframe['SG_UF_NOT'] == uf_id]

print('Número de colunas:', len(raw_dataframe.columns))
print('Número de linhas:', len(raw_dataframe))

### Agrupamento de variáveis

#### Relação com agressor

In [ ]:
def get_agressor_relationship(dataframe):

    '''
    Retorna uma lista das relações das vítimas com os agressores.

    Ex: ['Pai', 'Desconhecido', ..., 'N/A']
    '''

    relationship_dict = {
        'REL_PAI': 'Pai',
        'REL_MAE': 'Mãe',
        'REL_PAD': 'Padrasto',
        'REL_CONJ': 'Cônjugue',
        'REL_EXCON': 'Ex-cônjuge',
        'REL_NAMO': 'Namorado/a',
        'REL_EXNAM': 'Ex-namorado/a',
        'REL_FILHO': 'Filho/a',
        'REL_DESCO': 'Desconhecido',
        'REL_IRMAO': 'Irmão/ã',
        'REL_CONHEC': 'Amigo/a',
        'REL_CUIDA': 'Cuidador/a',
        'REL_PATRAO': 'Chefe',
        'REL_INST': 'Relação Institucional',
        'REL_POL': 'Policial/Agente da lei',
        'REL_PROPRI': 'Próprio/a',
        'REL_OUTROS': 'Outra pessoa',
        'REL_MAD': 'Madrasta'
    }

    agressor_relationship = []
    relationship_keys     = relationship_dict.keys()

    for index in tqdm(dataframe.index):

        case = dataframe.loc[index]

        found_rel = False

        for key in relationship_keys:

            relantionship = relationship_dict[key]

            if case[key] == 1:

                agressor_relationship.append(relantionship)
                found_rel = True

                break

        if not found_rel:

            agressor_relationship.append('N/A')

    return agressor_relationship

def preprocess_relationship(value):
    
    if value == '*':
        return 0
    
    elif value == '':
        return 0

    else:
        
        try:
            return np.int8(value)
        
        except:
            
            try:
                return np.int8(float(value))
            
            except: # NaN variables
                return 0

In [ ]:
%%time

# Corrigindo tipo das variáveis de relação

relationship_columns = ['REL_PAI','REL_MAE','REL_PAD','REL_CONJ','REL_EXCON',
                        'REL_NAMO','REL_EXNAM','REL_FILHO','REL_DESCO','REL_IRMAO',
                        'REL_CONHEC','REL_CUIDA','REL_PATRAO','REL_INST','REL_POL',
                        'REL_PROPRI','REL_OUTROS','REL_MAD']


for column in relationship_columns:

    # Convertendo valores para inteiros 
    raw_dataframe[column] = raw_dataframe[column].apply(preprocess_relationship)

In [ ]:
%%time

raw_dataframe['AGRESSOR'] = get_agressor_relationship(raw_dataframe)

In [ ]:
# Retirando dados omitidos
# df  = raw_dataframe[raw_dataframe['AGRESSOR'] != 'N/A']

fig = px.histogram(raw_dataframe, x="AGRESSOR", title=f'Relação: Vítima x Agressor ({len(raw_dataframe)} casos)')
fig.show()

### Tipo de violência

In [ ]:
def get_violence(dataframe):

    '''
    Retorna uma lista do tipo de violência sofrida pela vítima.

    Ex: ['Pai', 'Desconhecido', ..., 'N/A']
    '''
    
    violence_dict = {
        'VIOL_FISIC': 'Física',
        'VIOL_PSICO': 'Psicológica',
        'VIOL_TORT': 'Tortura',
        'VIOL_SEXU': 'Sexual',
        'VIOL_TRAF': 'Tráfico humano',
        'VIOL_FINAN': 'Financeira/econômica',
        'VIOL_NEGLI': 'Negligência/abandono',
        'VIOL_INFAN': 'Trabalho infantil',
        'VIOL_LEGAL': 'Legal',
        'VIOL_OUTR': 'Outra',
    }

    type_violence  = []
    violence_keys  = violence_dict.keys()

    for index in tqdm(dataframe.index):

        case = dataframe.loc[index]

        found_violence = False

        for key in violence_keys:

            violence = violence_dict[key]

            if case[key] == 1:

                type_violence.append(violence)
                found_violence = True

                break

        if not found_violence:

            type_violence.append('N/A')

    return type_violence

def preprocess_violence(value):
    
    if value == '*':
        return 0
    
    elif value == '':
        return 0

    else:
        
        try:
            return np.int8(value)
        
        except:
            
            try:
                return np.int8(float(value))
            
            except: # NaN variables
                return 0

In [ ]:
%%time

# Corrigindo tipo das variáveis de violencia

violence_columns = ['VIOL_FISIC','VIOL_PSICO','VIOL_TORT','VIOL_SEXU',
                    'VIOL_TRAF','VIOL_FINAN','VIOL_NEGLI','VIOL_INFAN',
                    'VIOL_LEGAL','VIOL_OUTR']


for column in violence_columns:

    # Convertendo valores para inteiros 
    raw_dataframe[column] = raw_dataframe[column].apply(preprocess_violence)

In [ ]:
%%time

raw_dataframe['VIOLENCE'] = get_violence(raw_dataframe)

In [ ]:
# Retirando dados omitidos
# df  = raw_dataframe[raw_dataframe['VIOLENCE'] != 'N/A']

fig = px.histogram(raw_dataframe, x="VIOLENCE", title=f'Tipo de violência sofrida ({len(raw_dataframe)} casos)')
fig.show()

### Raça do paciente

In [ ]:
def preprocess_race(value):
    
    try:
        return np.int8(value)

    except:

        try:
            return np.int8(float(value))

        except: # NaN variables
            return 9

In [ ]:
%%time

race_dict = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena',
    9: 'N/A'
}

raw_dataframe.CS_RACA = raw_dataframe.CS_RACA.apply(preprocess_race)
raw_dataframe.CS_RACA = raw_dataframe.CS_RACA.apply(lambda race_key: race_dict[race_key])

In [ ]:
# Retirando dados omitidos
# df  = raw_dataframe[raw_dataframe['CS_RACA'] != 'N/A']

fig = px.histogram(raw_dataframe, x="CS_RACA", title=f'Raças ({len(raw_dataframe)} casos)')
fig.show()

### Data

In [ ]:
def preprocess_year(dataframe):
    
    dt_year  = []
    dt_month = []
    dt_day   = []
    
    for index in tqdm(dataframe.index):
        
        row = dataframe.loc[index]
        
        date = row.DT_NOTIFIC
        
        try:
            date_list = date.split('-')
            
            dt_year.append(int(date_list[0]))
            dt_month.append(int(date_list[1]))
            dt_day.append(int(date_list[2]))
        except:
            
            dt_year.append(None)
            dt_month.append(None)
            dt_day.append(None)
            
    
    dataframe['DT_YEAR']  = dt_year
    dataframe['DT_MONTH'] = dt_month
    dataframe['DT_DAY']   = dt_day
    
    return dataframe

In [ ]:
%%time

raw_dataframe = preprocess_year(raw_dataframe)

### Idade

In [ ]:
def preprocess_age(dataframe):
    
    ages = []
    
    for index in tqdm(dataframe.index):
        
        row = dataframe.loc[index]
        
        born_date_str = row.DT_NASC
        
        try:
            born_date = datetime.strptime(born_date_str, '%Y-%m-%d')
            
            age = row.DT_YEAR - born_date.year
        except:
            
            age = None
            
        ages.append(age)
            
    
    dataframe['AGE']  = ages
    
    return dataframe

In [ ]:
%%time

raw_dataframe = preprocess_age(raw_dataframe)

### Escolaridade

In [ ]:
def preprocess_escolaridade(value):
    
    try:
        return np.int8(value)

    except:

        try:
            return np.int8(float(value))

        except: # NaN variables
            return 0

In [ ]:
%%time

escolaridade_dict = {
    1: 'EF I incompleto',
    2: 'EF I completo',
    3: 'EF II incompleto',
    4: 'EF II completo',
    5: 'EM incompleto',
    6: 'EM completo',
    7: 'ES incompleto',
    8: 'ES completo',
    9: None,
    10: None,
    0: None
}

raw_dataframe.CS_ESCOL_N   = raw_dataframe.CS_ESCOL_N.apply(preprocess_escolaridade)
raw_dataframe['ESCOLARIDADE'] = raw_dataframe.CS_ESCOL_N.apply(lambda escolaridade_key: escolaridade_dict[escolaridade_key])

### Ocupações

In [ ]:
cbo_ocupacoes    = pd.read_csv('./../data/cbo_ocupacoes.csv')
cbo_familia      = pd.read_csv('./../data/cbo_familia.csv')
cbo_grande_grupo = pd.read_csv('./../data/cbo_grande_grupo.csv')

cbo_ocupacoes['Código'] = cbo_ocupacoes['Código'].apply(lambda x:str(x))
cbo_familia['Código'] = cbo_familia['Código'].apply(lambda x:str(x))
cbo_grande_grupo['Código'] = cbo_grande_grupo['Código'].apply(lambda x:str(x))

CBO_OCUPACOES = cbo_ocupacoes.set_index('Código')
CBO_FAMILIA = cbo_familia.set_index('Código')
CBO_GRANDE_GRUPO = cbo_grande_grupo.set_index('Código')

In [6]:
def preprocess_ocupacao(value):
    
    try:
        return str(int(float(value)))

    except:

        return '*'
        
def process_cbo_ocupacao(cbo_cod):
    
    try:
        data  = CBO_OCUPACOES.loc[cbo_cod]
        value = data['Título']
    except:
        value = None
    
    return value

def process_cbo_familia(cbo_cod):
    
    cod = cbo_cod[:4]
    try:
        data  = CBO_FAMILIA.loc[cod]
        value = data['Título']
    except:
        cod = cbo_cod[:3]
        try:
            data  = CBO_FAMILIA.loc[cod]
            value = data['Título']
        except:
            value = None
    
    return value

def process_cbo_grande_grupo(cbo_cod):
    
    cod = cbo_cod[0]
    try:
        data  = CBO_GRANDE_GRUPO.loc[cod]
        value = data['Título']
    except:
        value = None
    
    return value

In [ ]:
%%time

raw_dataframe['ID_OCUPA_N']   = raw_dataframe.head()['ID_OCUPA_N'].apply(preprocess_ocupacao)
raw_dataframe['CBO_OCUPACAO'] = raw_dataframe.head()['ID_OCUPA_N'].apply(process_cbo_ocupacao)
raw_dataframe['CBO_FAMILIA']  = raw_dataframe.head()['ID_OCUPA_N'].apply(process_cbo_familia)
raw_dataframe['CBO_GRUPO']    = raw_dataframe.head()['ID_OCUPA_N'].apply(process_cbo_grande_grupo)

### Adicionando dados municipais

In [ ]:
municipios = pd.read_csv('./../data/municipios.csv')

In [ ]:
municipios.head()

In [ ]:
def add_municipios_data(dataframe, municipios_df):
    
    municipios_cols = list(municipios_df.columns)
    municipios_cols = [f'MUN_{col.upper()}' for col in municipios_cols]
    
    for col in municipios_cols:
        dataframe[col] = [None]*len(dataframe)
    
    for index in tqdm(dataframe.index):
        
        case     = dataframe.loc[index]
        sinan_id = case.ID_MUNICIP
        
        mun_data = municipios_df.loc[sinan_id]
        
        for col in municipios_cols:
            
            dataframe.at[index, col] = mun_data[col.split('MUN_')[1].lower()]
            
    return dataframe

In [ ]:
%%time

raw_dataframe = add_municipios_data(raw_dataframe, municipios.set_index('sinan_id'))

In [ ]:
raw_dataframe.head()

### Denuncias (per capita) por municipio

In [ ]:
denuncias = pd.DataFrame()

denuncias['count'] = raw_dataframe.groupby('MUN_NOME').count()['TP_NOT']

lat_arr = []
lng_arr = []
pop_arr = []

municipios_df = municipios.set_index('nome')

for mun_nome in tqdm(denuncias.index):
    
    data = municipios_df.loc[mun_nome]
    
    lat_arr.append(data['latitude'])
    lng_arr.append(data['longitude'])
    pop_arr.append(data['pop_2015'])
    
denuncias['lat'] = lat_arr
denuncias['lng'] = lng_arr
denuncias['pop'] = pop_arr

denuncias = denuncias.reset_index()

In [ ]:
denuncias.head()

In [ ]:
text_array = denuncias['MUN_NOME'] + ' - ' + (denuncias['count']/denuncias['pop']).apply(lambda x : str(x))

fig = go.Figure(data=go.Scattergeo(
        lon = denuncias['lng'],
        lat = denuncias['lat'],
        text = text_array,
        mode = 'markers',
        marker = dict(
                color = denuncias['count']/denuncias['pop'],
                reversescale = True,
                opacity = 0.7,
                size = 5,
                colorbar = dict(
                    titleside = "right",
                    outlinecolor = "rgba(68, 68, 68, 0)",
                    ticks = "outside",
                    showticksuffix = "last",
                )
            )
        ))

fig.update_layout(
        title = 'Denuncias per capita por município (SINAN)',
        geo_scope='south america',
    )
fig.show()

In [ ]:
text_array = denuncias['MUN_NOME'] + ' - ' + (denuncias['count']).apply(lambda x : str(x))

fig = go.Figure(data=go.Scattergeo(
        lon = denuncias['lng'],
        lat = denuncias['lat'],
        text = text_array,
        mode = 'markers',
        marker = dict(
                color = denuncias['count'],
                reversescale = True,
                opacity = 0.7,
                size = 5,
                colorbar = dict(
                    titleside = "right",
                    outlinecolor = "rgba(68, 68, 68, 0)",
                    ticks = "outside",
                    showticksuffix = "last",
                )
            )
        ))

fig.update_layout(
        title = 'Denuncias por município (SINAN)',
        geo_scope='south america',
    )

fig.show()